In [1]:
import pandas as pd

dfwifi = pd.read_excel('wifi.xlsx')
dfcom = pd.read_excel('computational.xlsx')
dfener = pd.read_excel('energy.xlsx')


/var/folders/dy/s09mg6hn01dbnd5qb_w1sb4r0000gq/T/ipykernel_4059/2696071309.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
dfwifi=dfwifi.reset_index(drop=True)
dfcom=dfcom.reset_index(drop=True)
dfener=dfener.reset_index(drop=True)

In [3]:
# dfwifi = dfwifi.loc[dfwifi['MicrocellID'].isin(["M_201","M_146","M_108"])]

## Without replications

In [4]:
S_wifi = [0.3,0.2,0.1,0.1,0.1,0.2]


In [5]:
unique_wifi = dfwifi.MicrocellID.unique()

dic_wifi={}

for microcell in unique_wifi:
    dic_wifi["{}".format(microcell)] = dfwifi[dfwifi.MicrocellID==microcell]

len(dic_wifi)

263

In [6]:
Wifi_scores={}

for k in dic_wifi.keys():
    wifi_df = dic_wifi[k]
    wifi_df = wifi_df.reset_index(drop=True)
    list1=[]
    for i in range(0,wifi_df.shape[0]):
        list1.append(((wifi_df['Latency'][i]*S_wifi[0])+(wifi_df['Throughput'][i]*S_wifi[1])+(wifi_df['Packet Loss'][i]*S_wifi[2])+(wifi_df['Jitter'][i]*S_wifi[3])+(wifi_df['Prioritization'][i]*S_wifi[4])+(wifi_df['Reliability'][i]*S_wifi[5]))/10)
    wifi_df['TS'] = list1
    Wifi_scores[k] = wifi_df
    

In [7]:
Wifi_scores


{'M_100':     ProviderID_V1 ProviderID_V2 ConsumerID ServiceID MicrocellID  \
 0         P_20020       p_40050     C_6001   S_86468       M_100   
 1         P_20023       p_40053     C_6004   S_86471       M_100   
 2         P_20133       p_40160     C_6116   S_86583       M_100   
 3         P_20163       p_40186     C_6149   S_86616       M_100   
 4         P_20233       p_40245     C_6223   S_86690       M_100   
 ..            ...           ...        ...       ...         ...   
 107       P_20233       p_40245    C_11758   S_92225       M_100   
 108       P_24752       p_40867    C_11759   S_92226       M_100   
 109       P_24753       p_40080    C_11760   S_92227       M_100   
 110       P_24754       p_42161    C_11761   S_92228       M_100   
 111       P_24774       p_40141    C_11785   S_92252       M_100   
 
     Superprovider   Timestamp      Function  Latency  Throughput  Packet Loss  \
 0           M_100  2017-04-04  WiFi Hotspot       10          10           10 

In [8]:
Data_visual=[]
for k in Wifi_scores.keys():
    TS_1=""
    per_provider_TS=""
    df1 = Wifi_scores[k]
    TS_1=df1["TS"].mean()
    unique_providers = df1.ProviderID_V1.unique()
    for provider in unique_providers:
        val = df1.loc[df1['ProviderID_V1'] == provider, 'TS'].mean()
        per_provider_TS = per_provider_TS + provider+ " "+ "trust Score : " + str("{:.4f}".format(val))+"<br>"

    
    # lat = df1["Latitude"].mean()
    # lat = long = df1["Longitude"].mean()
    lat = df1.loc[df1['MicrocellID'] == k, 'Latitude'].mean()
    long = df1.loc[df1['MicrocellID'] == k, 'Longitude'].mean()
    Data_visual.append([k,TS_1,lat,long,per_provider_TS])

In [9]:
dfnew = pd.DataFrame(Data_visual, columns=['Microcell', 'TS1','Latitude','Longitude','provider_scores'])
 

print(dfnew)

    Microcell       TS1   Latitude   Longitude  \
0       M_100  0.955536 -33.874168  151.222695   
1       M_101  0.966667 -33.893904  151.228969   
2       M_102  0.954977 -33.877432  151.213558   
3       M_103  0.961989 -33.905722  151.233922   
4       M_104  0.945122 -33.898387  151.268067   
..        ...       ...        ...         ...   
258     M_358  0.845000 -33.827443  151.146658   
259     M_359  0.990000 -33.972307  151.131506   
260     M_360  1.000000 -33.845383  151.239645   
261     M_361  1.000000 -33.984147  151.148158   
262     M_362  0.960000 -33.896685  151.269239   

                                       provider_scores  
0    P_20020 trust Score : 0.9900<br>P_20023 trust ...  
1    P_20021 trust Score : 1.0000<br>P_20510 trust ...  
2    P_20022 trust Score : 0.9700<br>P_20029 trust ...  
3    P_20024 trust Score : 0.9400<br>P_20035 trust ...  
4    P_20025 trust Score : 1.0000<br>P_20026 trust ...  
..                                                 ...  


In [10]:
# dfnew.to_excel('test_1.xlsx')

In [11]:
import folium
m = folium.Map((-33.841097, 151.278535), zoom_start=12)




for i in range(0,dfnew.shape[0]):
    html = """
    <html>\n 
    <head>\n
    <title> \n  \ </title>\n</head> 
    <body> 
    <h3> Microcell : """ + str(dfnew['Microcell'][i]) +"""</h3> \n 
  <h4>Wifi Score : """ + str("{:.4f}".format(dfnew['TS1'][i])) + """</h4> \n 
 <h4>Provider Scores</h4> \n 
  <h4> """ + str(dfnew['provider_scores'][i]) + """</h4> \n 
    </body></html>"""

    iframe = folium.IFrame(html,width=400,height=200)

    popup = folium.Popup(iframe,max_width=400)
    
    folium.Marker(location=[dfnew['Latitude'][i], dfnew['Longitude'][i]],icon=folium.Icon(color="green"),popup=popup).add_to(m)

/Users/tlok9884/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [12]:
m

## With replication

In [1]:
import pandas as pd
dfwifi_rep = pd.read_excel('wifi.xlsx')


/var/folders/dy/s09mg6hn01dbnd5qb_w1sb4r0000gq/T/ipykernel_4082/400315765.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
dfwifi_rep=dfwifi_rep.reset_index(drop=True)


In [3]:
unique_wifi = dfwifi_rep.MicrocellID.unique()

dic_wifi={}

for microcell in unique_wifi:
    dic_wifi["{}".format(microcell)] = dfwifi_rep[dfwifi_rep.MicrocellID==microcell]

len(dic_wifi)

263

In [4]:
data_wifi={}
for key, df_sel in dic_wifi.items():
    df_sel=df_sel.reset_index(drop=True)
    for i in range (0,df_sel.shape[0]):
        dflag = dfwifi_rep[dfwifi_rep.ProviderID_V1==df_sel['ProviderID_V1'][i]]
        dflag = dflag.reset_index(drop=True)
        for j in range (0,dflag.shape[0]):
            if(str(dflag['MicrocellID'][j])!=key):
                df_sel.loc[len(df_sel.index)] = [dflag['ProviderID_V1'][j],dflag['ProviderID_V2'][j],dflag['ConsumerID'][j],dflag['ServiceID'][j],dflag['MicrocellID'][j],dflag['Superprovider'][j],dflag['Timestamp'][j],dflag['Function'][j],dflag['Latency'][j],dflag['Throughput'][j],dflag['Packet Loss'][j],dflag['Jitter'][j],dflag['Prioritization'][j],dflag['Reliability'][j],dflag['Latitude'][j],dflag['Longitude'][j],dflag['Area'][j],'Received']  
            else:
                df_sel= df_sel
    data_wifi[key] = df_sel

In [5]:
data_wifi.keys()

dict_keys(['M_100', 'M_101', 'M_102', 'M_103', 'M_104', 'M_105', 'M_106', 'M_107', 'M_108', 'M_109', 'M_110', 'M_111', 'M_112', 'M_113', 'M_114', 'M_115', 'M_116', 'M_117', 'M_118', 'M_119', 'M_120', 'M_121', 'M_122', 'M_123', 'M_124', 'M_125', 'M_126', 'M_127', 'M_128', 'M_129', 'M_130', 'M_131', 'M_132', 'M_133', 'M_134', 'M_135', 'M_136', 'M_137', 'M_138', 'M_139', 'M_140', 'M_141', 'M_142', 'M_143', 'M_144', 'M_145', 'M_146', 'M_147', 'M_148', 'M_149', 'M_150', 'M_151', 'M_152', 'M_153', 'M_154', 'M_155', 'M_156', 'M_157', 'M_158', 'M_159', 'M_160', 'M_161', 'M_162', 'M_163', 'M_164', 'M_165', 'M_166', 'M_167', 'M_168', 'M_169', 'M_170', 'M_171', 'M_172', 'M_173', 'M_174', 'M_175', 'M_176', 'M_177', 'M_178', 'M_179', 'M_180', 'M_181', 'M_182', 'M_183', 'M_184', 'M_185', 'M_186', 'M_187', 'M_188', 'M_189', 'M_190', 'M_191', 'M_192', 'M_193', 'M_194', 'M_195', 'M_196', 'M_197', 'M_198', 'M_199', 'M_200', 'M_201', 'M_202', 'M_203', 'M_204', 'M_205', 'M_206', 'M_207', 'M_208', 'M_209',

In [6]:
len(data_wifi)

263

In [7]:
S_wifi = [0.3,0.2,0.1,0.1,0.1,0.2]

In [8]:
Wifi_scores_replicated={}

for k in data_wifi.keys():
    wifi_df = data_wifi[k]
    wifi_df = wifi_df.reset_index(drop=True)
    list1=[]
    for i in range(0,wifi_df.shape[0]):
        list1.append(((wifi_df['Latency'][i]*S_wifi[0])+(wifi_df['Throughput'][i]*S_wifi[1])+(wifi_df['Packet Loss'][i]*S_wifi[2])+(wifi_df['Jitter'][i]*S_wifi[3])+(wifi_df['Prioritization'][i]*S_wifi[4])+(wifi_df['Reliability'][i]*S_wifi[5]))/10)
    wifi_df['TS'] = list1
    Wifi_scores_replicated[k] = wifi_df

In [9]:
Data_visual=[]
for k in Wifi_scores_replicated.keys():
    TS_1=""
    per_provider_TS=""
    df1 = Wifi_scores_replicated[k]
    TS_1=df1["TS"].mean()
    unique_providers = df1.ProviderID_V1.unique()
    for provider in unique_providers:
        val = df1.loc[df1['ProviderID_V1'] == provider, 'TS'].mean()
        per_provider_TS = per_provider_TS + provider+ " "+ "trust Score : " + str("{:.4f}".format(val))+"<br>"

    
    # lat = df1["Latitude"].mean()
    # long = df1["Longitude"].mean()
    lat = df1.loc[df1['MicrocellID'] == k, 'Latitude'].mean()
    long = df1.loc[df1['MicrocellID'] == k, 'Longitude'].mean()
    Data_visual.append([k,TS_1,lat,long,per_provider_TS])

In [10]:
dfnew = pd.DataFrame(Data_visual, columns=['Microcell', 'TS1','Latitude','Longitude','provider_scores'])
 

print(dfnew)

    Microcell       TS1   Latitude   Longitude  \
0       M_100  0.933834 -33.874168  151.222695   
1       M_101  0.967500 -33.893904  151.228969   
2       M_102  0.936192 -33.877432  151.213558   
3       M_103  0.921038 -33.905722  151.233922   
4       M_104  0.909962 -33.898387  151.268067   
..        ...       ...        ...         ...   
258     M_358  0.845000 -33.827443  151.146658   
259     M_359  0.990000 -33.972307  151.131506   
260     M_360  1.000000 -33.845383  151.239645   
261     M_361  1.000000 -33.984147  151.148158   
262     M_362  0.960000 -33.896685  151.269239   

                                       provider_scores  
0    P_20020 trust Score : 0.9900<br>P_20023 trust ...  
1    P_20021 trust Score : 1.0000<br>P_20510 trust ...  
2    P_20022 trust Score : 0.9700<br>P_20029 trust ...  
3    P_20024 trust Score : 0.9400<br>P_20035 trust ...  
4    P_20025 trust Score : 1.0000<br>P_20026 trust ...  
..                                                 ...  


In [11]:
# dfnew.to_excel('test_rep_1.xlsx')

In [12]:
import folium
m1 = folium.Map((-33.841097, 151.278535), zoom_start=12)




for i in range(0,dfnew.shape[0]):
    html = """
    <html>\n 
    <head>\n
    <title> \n  \ </title>\n</head> 
    <body> 
    <h3> Microcell : """ + str(dfnew['Microcell'][i]) +"""</h3> \n 
  <h4>Wifi Score : """ + str("{:.4f}".format(dfnew['TS1'][i])) + """</h4> \n 
 <h4>Provider Scores</h4> \n 
  <h4> """ + str(dfnew['provider_scores'][i]) + """</h4> \n 
    </body></html>"""

    iframe = folium.IFrame(html,width=400,height=200)

    popup = folium.Popup(iframe,max_width=400)
    
    folium.Marker(location=[dfnew['Latitude'][i], dfnew['Longitude'][i]],icon=folium.Icon(color="green"),popup=popup).add_to(m1)

/Users/tlok9884/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [13]:
m1

In [ ]:
unique_com = dfcom.MicrocellID.unique()

dic_com={}

for microcell in unique_com:
    dic_com["{}".format(microcell)] = dfcom[dfcom.MicrocellID==microcell]


data_com={}
for key, df_sel in dic_com.items():
    df_sel=df_sel.reset_index(drop=True)
    for i in range (0,df_sel.shape[0]):
        dflag = dfcom[dfcom.ProviderID_V2==df_sel['ProviderID_V2'][i]]
        dflag = dflag.reset_index(drop=True)
        for j in range (0,dflag.shape[0]):
            if(str(dflag['MicrocellID'][j])!=key):
                df_sel.loc[len(df_sel.index)] = [dflag['ProviderID_V1'][j],dflag['ProviderID_V2'][j],dflag['ConsumerID'][j],dflag['ServiceID'][j],dflag['MicrocellID'][j],dflag['Superprovider'][j],dflag['Timestamp'][j],dflag['Function'][j],dflag['Speed'][j],dflag['Accuracy'][j],dflag['Availability'][j],dflag['Reliability'][j],dflag['Packetloss'][j],dflag['Latitude'][j],dflag['Longitude'][j],dflag['Area'][j],'Received']  
            else:
                df_sel= df_sel
    data_com[key] = df_sel

data_com.keys()


In [ ]:
unique_com = dfener.MicrocellID.unique()

dic_ener={}

for microcell in unique_com:
    dic_ener["{}".format(microcell)] = dfener[dfener.MicrocellID==microcell]


data_ener={}
for key, df_sel in dic_ener.items():
    df_sel=df_sel.reset_index(drop=True)
    for i in range (0,df_sel.shape[0]):
        dflag = dfener[dfener.ProviderID_V2==df_sel['ProviderID_V2'][i]]
        dflag = dflag.reset_index(drop=True)
        for j in range (0,dflag.shape[0]):
            if(str(dflag['MicrocellID'][j])!=key):
                df_sel.loc[len(df_sel.index)] = [dflag['ProviderID_V1'][j],dflag['ProviderID_V2'][j],dflag['ConsumerID'][j],dflag['ServiceID'][j],dflag['MicrocellID'][j],dflag['Superprovider'][j],dflag['Timestamp'][j],dflag['Function'][j],dflag['Reliability'][j],dflag['Availability'][j],dflag['Coverage'][j],dflag['Latitude'][j],dflag['Longitude'][j],dflag['Area'][j],'Received']  
            else:
                df_sel= df_sel
    data_ener[key] = df_sel

data_ener.keys()

In [ ]:
keys = data_wifi.keys()

In [ ]:
keys

In [ ]:
for i in keys:
    print(i)
    print(data_wifi[i])
    print(data_com[i])
    if i in data_ener.keys():
        print(data_ener[i])

    

In [ ]:
S_wifi = [0.3,0.2,0.1,0.1,0.1,0.1]
S_com = [0.3,0.2,0.1,0.1,0.2]
S_ener = [0.4,0.3,0.3]

In [ ]:
data_wifi_new={}
data_com_new={}
data_ener_new={}


for k in keys:
    wifi_df = data_wifi[k]
    wifi_df = wifi_df.reset_index(drop=True)
    for i in range(0,wifi_df.shape[0]):
        wifi_df['TS'] = ((wifi_df['Latency'][i]*S_wifi[0])+(wifi_df['Throughput'][i]*S_wifi[1])+(wifi_df['Packet Loss'][i]*S_wifi[2])+(wifi_df['Jitter'][i]*S_wifi[3])+(wifi_df['Prioritization'][i]*S_wifi[4])+(wifi_df['Reliability'][i]*S_wifi[5]))/10
    data_wifi_new[k] = wifi_df
    com_df = data_com[k]
    com_df = com_df.reset_index(drop=True)
    for i in range(0,com_df.shape[0]):
        com_df['TS'] = ((com_df['Speed'][i]*S_com[0])+(com_df['Accuracy'][i]*S_com[1])+(com_df['Availability'][i]*S_com[2])+(com_df['Reliability'][i]*S_com[3])+(com_df['Packetloss'][i]*S_com[4]))/10
    data_com_new[k] = com_df
    if k in data_ener.keys():
        ener_df = data_ener[k]
        ener_df = ener_df.reset_index(drop=True)
        for i in range(0,ener_df.shape[0]):
            ener_df['TS'] = ((ener_df['Reliability'][i]*S_ener[0])+(ener_df['Availability'][i]*S_ener[1])+(ener_df['Coverage'][i]*S_ener[2]))/10
        data_ener_new[k] = ener_df

    # print(data_wifi[i])
    # print(data_com[i])
    # 
    #     print(data_ener[i])

In [ ]:
Data_visual=[]
for k in keys:
    TS_1=""
    TS_2=""
    TS_3=""
    OS=""
    sum1=0
    df1 = data_wifi_new[k]
    TS_1=df1["TS"].mean()
    df2 = data_com_new[k]
    TS_2=df2["TS"].mean()
    if k in data_ener_new.keys():
        df3 = data_ener_new[k]
        TS_3=df3["TS"].mean()
        sum1 = df1.shape[0]+df2.shape[0]+df3.shape[0]
        OS = (TS_1*(df1.shape[0]/sum1))+(TS_2*(df2.shape[0]/sum1))+(TS_3*(df3.shape[0]/sum1))


    else:
        TS_3="Null"
        sum1 = df1.shape[0]+df2.shape[0]
        OS = (TS_1*(df1.shape[0]/sum1))+(TS_2*(df2.shape[0]/sum1))
    
    lat = df1["Latitude"].mean()
    long = df1["Longitude"].mean()
    Data_visual.append([k,TS_1,TS_2,TS_3,OS,lat,long])

In [ ]:
dfnew = pd.DataFrame(Data_visual, columns=['Microcell', 'TS1','TS2','TS3','Overall Score','Latitude','Longitude'])
 

print(dfnew)

In [ ]:
import folium
m = folium.Map((-33.841097, 151.278535), zoom_start=12)




for i in range(0,dfnew.shape[0]):
    html = """
    <html>\n 
    <head>\n
    <title> \n  \ </title>\n</head> 
    <body> 
    <h3> Microcell : """ + str(dfnew['Microcell'][i]) +"""</h3> \n 
    <h4>Overall Score : """ + str(round(dfnew['Overall Score'][i],4)) + """</h4> \n 
  <h4>Wifi Score : """ + str(round(dfnew['TS1'][i],4)) + """</h4> \n 
       <h4>Computational Score : """ + str(round(dfnew['TS2'][i],4)) + """</h4> \n
        <h4>Energy Score : """ + str(dfnew['TS3'][i]) + """</h4> \n
    </body></html>"""

    iframe = folium.IFrame(html,width=400,height=200)

    popup = folium.Popup(iframe,max_width=400)
    
    folium.Marker(location=[dfnew['Latitude'][i], dfnew['Longitude'][i]],icon=folium.Icon(color="green"),popup=popup).add_to(m)

In [ ]:
m

In [ ]:
data=[]
for key, value in dic.items():
    df1 = value
    df1=df1.reset_index(drop=True)
    test1 =" "
    received=""
    for i in range (0,df1.shape[0]):
        # main_microcell = str(df1['Microcell'][i])
        test1= test1+"PID: "+df1['ProviderID_V1'][i]+" CID: " +str(df1['ConsumerID'][i])+" t: " +str(df1['Timestamp'][i])+" m: " +str(df1['MicrocellID'][i])+" Scores : " +str(df1['Latency'][i] ) + " " +str(df1['Throughput'][i] ) +" " +str(df1['Packet Loss'][i] ) +" " +str(df1['Jitter'][i] ) +" " +str(df1['Prioritization'][i] ) +" " +str(df1['Reliability'][i]) +" f: " +str(df1['Function'][i])  +" <br> "
        dflag = df[df.ProviderID_V1==df1['ProviderID_V1'][i]]
        dflag = dflag.reset_index(drop=True)
        for j in range (0,dflag.shape[0]):
            if(str(dflag['MicrocellID'][j])!=key):
                received= received+"PID: "+dflag['ProviderID_V1'][j]+" CID: " +str(dflag['ConsumerID'][j])+" t: " +str(dflag['Timestamp'][j])+" m: " +str(dflag['MicrocellID'][j])+" Scores : " +str(dflag['Latency'][j] ) + " " +str(dflag['Throughput'][j] ) +" " +str(dflag['Packet Loss'][j] ) +" " +str(dflag['Jitter'][j] ) +" " +str(dflag['Prioritization'][j] ) +" " +str(dflag['Reliability'][j]) +" f: " +str(dflag['Function'][j])  +" <br> "
            else:
                received= received
    lat = df1['Latitude'][0]
    long = df1['Longitude'][0]
    data.append([key,test1,received,lat,long])

Calculation of TS (Trust Score per TIR)


In [ ]:
for key1, val in data_ener.items():

    name = "./out/out_"+str(key1)+".xlsx"
    print(name)
    val.to_excel(name)

In [ ]:
df1 = df.loc[df['MicrocellID'].isin(["M_128","M_102","M_100","M_137","M_136","M_115","M_117","M_103","M_155","M_140","M_147","M_119"])]

In [ ]:
df1 = df

In [ ]:
df1=df1.reset_index(drop=True)
df1

In [ ]:
refuniquenames = df1.MicrocellID.unique()
refuniquenames

dic={}

for microcell in refuniquenames:
    dic["{}".format(microcell)] = df[df.MicrocellID==microcell]

dic

In [ ]:
new_concat_df = pd.DataFrame( columns=['ProviderID_V1','ProviderID_V2','ConsumerID','ServiceID','MicrocellID','Superprovider','Timestamp','Function','Latency','Throughput','Packet Loss','Jitter','Prioritization','Reliability','Crowdsourcing Rating','Latitude','Longitude','Area'])

for key, df_sel in dic.items():
    new_concat_df = pd.concat([new_concat_df, df_sel], axis=0)

In [ ]:
new_concat_df

In [ ]:
data={}
for key, df_sel in dic.items():
    df_sel=df_sel.reset_index(drop=True)
    for i in range (0,df_sel.shape[0]):
        dflag = new_concat_df[new_concat_df.ProviderID_V2==df_sel['ProviderID_V2'][i]]
        dflag = dflag.reset_index(drop=True)
        new_data_list=[]
        for j in range (0,dflag.shape[0]):
            if(str(dflag['MicrocellID'][j])!=key):
                df_sel.loc[len(df_sel.index)] = [dflag['ProviderID_V1'][j],dflag['ProviderID_V2'][j],dflag['ConsumerID'][j],dflag['ServiceID'][j],dflag['MicrocellID'][j],dflag['Superprovider'][j],dflag['Timestamp'][j],dflag['Function'][j],dflag['Latency'][j],dflag['Throughput'][j],dflag['Packet Loss'][j],dflag['Jitter'][j],dflag['Prioritization'][j],dflag['Reliability'][j],dflag['Crowdsourcing Rating'][j],dflag['Latitude'][j],dflag['Longitude'][j],dflag['Area'][j],'Received']  
            else:
                df_sel= df_sel
    data[key] = df_sel
    # data.append(df_sel)
                


In [ ]:
data.keys()

In [ ]:
for key1, val in data.items():

    name = "./out/out_"+str(key1)+".xlsx"
    print(name)
    val.to_excel(name)

In [ ]:
data=[]
for key, value in dic.items():
    df1 = value
    df1=df1.reset_index(drop=True)
    test1 =" "
    received=""
    for i in range (0,df1.shape[0]):
        # main_microcell = str(df1['Microcell'][i])
        test1= test1+"PID: "+df1['ProviderID_V1'][i]+" CID: " +str(df1['ConsumerID'][i])+" t: " +str(df1['Timestamp'][i])+" m: " +str(df1['MicrocellID'][i])+" Scores : " +str(df1['Latency'][i] ) + " " +str(df1['Throughput'][i] ) +" " +str(df1['Packet Loss'][i] ) +" " +str(df1['Jitter'][i] ) +" " +str(df1['Prioritization'][i] ) +" " +str(df1['Reliability'][i]) +" f: " +str(df1['Function'][i])  +" <br> "
        dflag = df[df.ProviderID_V1==df1['ProviderID_V1'][i]]
        dflag = dflag.reset_index(drop=True)
        for j in range (0,dflag.shape[0]):
            if(str(dflag['MicrocellID'][j])!=key):
                received= received+"PID: "+dflag['ProviderID_V1'][j]+" CID: " +str(dflag['ConsumerID'][j])+" t: " +str(dflag['Timestamp'][j])+" m: " +str(dflag['MicrocellID'][j])+" Scores : " +str(dflag['Latency'][j] ) + " " +str(dflag['Throughput'][j] ) +" " +str(dflag['Packet Loss'][j] ) +" " +str(dflag['Jitter'][j] ) +" " +str(dflag['Prioritization'][j] ) +" " +str(dflag['Reliability'][j]) +" f: " +str(dflag['Function'][j])  +" <br> "
            else:
                received= received
    lat = df1['Latitude'][0]
    long = df1['Longitude'][0]
    data.append([key,test1,received,lat,long])

Tampering test

In [ ]:
testdf = data['M_100']

In [ ]:
testdf

In [ ]:
unique_providers = testdf['ProviderID_V2'].unique()

In [ ]:
provider_amont = 20
TIR_amont = 50
attribute_amont = 20
each_attribute = 30



In [ ]:
len(unique_providers)

In [ ]:
number_of_providers = round((provider_amont*len(unique_providers))/100)
number_of_providers

In [ ]:
import random

tampereing_providers = random.sample(sorted(unique_providers), number_of_providers)

tampereing_providers

In [ ]:
for k in range (0,testdf.shape[0]):
    
    print(testdf['ProviderID_V2'][k])

In [ ]:
df_after_tampering = pd.DataFrame( columns=['ProviderID_V1','ProviderID_V2','ConsumerID','ServiceID','MicrocellID','Superprovider','Timestamp','Function','Latency','Throughput','Packet Loss','Jitter','Prioritization','Reliability','Crowdsourcing Rating','Latitude','Longitude','Area','Source','Tamp'])

for provider in unique_providers:
    provider_frame = testdf[testdf['ProviderID_V2'] == provider]
    provider_frame = provider_frame.reset_index(drop=True)
    if provider in tampereing_providers:
        TIR_break = round((TIR_amont*provider_frame.shape[0])/100)
        # print(provider_frame)
        # print("loop")
        df_to_tam = provider_frame.iloc[: TIR_break,:]
        df_to_tam = df_to_tam.reset_index(drop=True)
        df_pass = provider_frame.iloc[TIR_break :, :]
        df_pass = df_pass.reset_index(drop=True)
        print(df_to_tam)
        print("9loop")
        if(df_to_tam.shape[0]!=0):
            for line in range(0,df_to_tam.shape[0]):
                # df_to_tam['Latency'][line] = 0
                df_to_tam['Latency'][line] = min(round(df_to_tam['Latency'][line]*(1+(each_attribute/100))),10)
                df_to_tam['Throughput'][line] = min(round(df_to_tam['Throughput'][line]*(1+(each_attribute/100))),10)
                df_to_tam['Packet Loss'][line] = min(round(df_to_tam['Packet Loss'][line]*(1+(each_attribute/100))),10)
                df_to_tam['Jitter'][line] = min(round(df_to_tam['Jitter'][line]*(1+(each_attribute/100))),10)
                df_to_tam['Prioritization'][line] = min(round(df_to_tam['Prioritization'][line]*(1+(each_attribute/100))),10)
                df_to_tam['Reliability'][line] = min(round(df_to_tam['Reliability'][line]*(1+(each_attribute/100))),10)

        df_to_tam['Tamp']='Y'
        df_after_tampering = pd.concat([df_after_tampering, df_to_tam], axis=0)
        df_pass['Tamp']='N'
        df_after_tampering = pd.concat([df_after_tampering, df_pass], axis=0)
    else:
        provider_frame['Tamp'] = 'N'
        df_after_tampering = pd.concat([df_after_tampering, provider_frame], axis=0)
        
# df_after_tampering

In [ ]:
df_after_tampering = df_after_tampering.reset_index(drop=True)

In [ ]:
df_after_tampering.to_excel("test_out.xlsx")

In [ ]:
x1 = 0.4
x2 = 0.6
x3 = 0.8

w1=0.6
w2 = 0.3
w3=0.1

In [ ]:
g = (x1+x2+x3)/3
k = (x1*w1+x2*w2+x3*w3)

In [ ]:
g

In [ ]:
k

In [ ]:
a1= [0.9,0.7,0.9,0.7]
a2= [0.6,0.8,0.9,0.7,0.6,0.8,0.9,0.7]
a3= [0.3,0.4]

In [ ]:
x1 = sum(a1)/len(a1)
x2 = sum(a2)/len(a2)
x3 = sum(a3)/len(a3)
print(x1)
print(x2)
print(x3)

In [ ]:
w1 = len(a1)/(len(a1)+len(a2)+len(a3))
w2 = len(a2)/(len(a1)+len(a2)+len(a3))
w3 = len(a3)/(len(a1)+len(a2)+len(a3))

In [ ]:
g = (x1+x2+x3)/3
k = (x1*w1+x2*w2+x3*w3)

print(g)
print(k)

In [ ]:
a=a1+a2+a3

In [ ]:
print(sum(a)/len(a))

In [ ]:
a

In [ ]:
a